In [ ]:
import numpy as np
import pandas as pd
import altair as alt
# disable row limit for plotting
alt.data_transformers.disable_max_rows()
# uncomment to ensure graphics display with pdf export
# alt.renderers.enable('mimetype')
alt.renderers.enable('default')

# Background 

Gender achievement gaps in education have been well-documented over the years -- studies consistently find boys outperforming girls on math tests and girls outperforming boys on reading and language tests. A particularly controversial [article](https://www.jstor.org/stable/1684489) was published in Science in 1980 arguing that this pattern was due to an 'innate' difference in ability (focusing on mathematics rather than on reading and language). Such views persisted in part because studying systematic patterns in achievement nationwide was a challenge due to differential testing standards across school districts and the general lack of availability of large-scale data.

It is only recently that data-driven research has begun to reveal socioeconomic drivers of achievement gaps. The [Standford Educational Data Archive](https://edopportunity.org/) (SEDA), a publicly available database on academic achievement and educational opportunity in U.S. schools, has supported this effort. The database is part of a broader initiave aiming to improve educational opportunity by enabling researchers and policymakers to identify systemic drivers of disparity.

> SEDA includes a range of detailed data on educational conditions, contexts, and outcomes in school districts and counties across the United States. It includes measures of academic achievement and achievement gaps for school districts and counties, as well as district-level measures of racial and socioeconomic composition, racial and socioeconomic segregation patterns, and other features of the schooling system.

The database standardizes average test scores for schools 10,000 U.S. school districts relative to national standards to allow comparability between school districts and across grade levels and years. The test score data come from the U.S. Department of Education. In addition, multiple data sources (American Community Survey and Common Core of Data) are integrated to provide district-level socioeconomic and demographic information.

A [study of the SEDA data published in 2018](https://cepa.stanford.edu/content/gender-achievement-gaps-us-school-districts) identified the following persistent patterns across grade levels 3 - 8 and school ears from 2008 through 2015:
* a consistent reading and language achievement gap favoring girls;
* *no* national math achievement gap on average; and
* local math achievement gaps that depend on the socioeconomic conditions of school districts.
You can read about the main findings of the study in this [brief NY Times article](https://www.nytimes.com/interactive/2018/06/13/upshot/boys-girls-math-reading-tests.html).

Below, I'll work with selected portions of the database. The full datasets can be downloaded [here](https://edopportunity.org/get-the-data/seda-archive-downloads/).

## Objectives

In this assignment, I'll explore achievement gaps in California school districts in 2018, replicating the findings described [in the NYT article](https://www.nytimes.com/interactive/2018/06/13/upshot/boys-girls-math-reading-tests.html) on California and using the most recent SEDA data. I'll practice the following:

* review of data documentation
* assessment of sampling design and scope of inference
* data tidying operations
    + slicing and filtering
    + merging multiple data frames
    + pivoting tables
    + renaming and reordering variables
* constructing exploratory graphics and visualizing trends
* data aggregations
* narrative summary of exploratory analysis

# Import and assessment of datasets

We'll work with test data and socioeconomic covariates aggregated to the school district level. These data are stored in two separate tables. Here, we'll examine them and review data documentation.

## Test score data

The first few rows of the test data are shown below. The columns are:

Column name | Meaning
---|---
`sedalea` | District ID
`grade` | Grade level
`stateabb` | State abbreviation
`sedaleaname` | District name
`subject` | Test subject
`cs_mn_...` | Estimated mean test score
`cs_mnse_...` | Standard error for estimated mean test score
`totgyb_...` | Number of individual tests used to estimate the mean score

In [ ]:
# import seda data
ca_main = pd.read_csv('../data/ca-main.csv')
ca_cov = pd.read_csv('../data/ca-cov.csv')

# preview test score data
ca_main.head(4)

The test score means for each district are named `cs_mn_...` with an abbreviation indicating subgroup (such as the mean score for all `cs_mean_all,` for boys `cs_mean_mal,` for white students `cs_mn_wht,` and so on). Notice that these are generally small-ish: decimal numbers between -0.5 and 0.5.

These means are *estimated* from several individual student tests and *standardized* relative to national averages. They represent the number of standard deviations by which a district mean differs from the national average. For instance, the value `cs_mn_all = 0.1` indicates that the district average is estimated to be 0.1 standard deviations higher than the national average on the corresponding test and at the corresponding grade level.

### Interpretation of test score values


We will be examining average math test scores for all 4th-grade students in the Acton-Agua Dulce Unified School District (the first row). The value `cs_mn_all = -0.367007` indicates that the district average is estimated to be 0.367 standard deviations lower than the national average on the corresponding test and at the corresponding grade level.


<!-- END QUESTION -->

## Covariate data

The first few rows of the covariate data are shown below. The column information is as follows:

Column name | Meaning
---|---
`sedalea` | District ID
`grade` | Grade level
`sedaleanm` | District name
`urban` | Indicator: is the district in an urban locale?
`suburb` | Indicator: is the district in a suburban locale?
`town` | Indicator: is the district in a town locale?
`rural` | Indicator: is the district in a rural locale?
`locale` | Description of district locale
Remaining variables | Demographic and socioeconomic measures

In [ ]:
ca_cov.head(3)

We will only be working with a handful of the demographic and socioeconomic measures, so we can put off getting acquainted with those until selecting a subset of variables.

### Data semantics

In the non-public data, the observational units are students; test scores are measured for each student. However, in the SEDA data we've imported, scores are *aggregated* to the district level by grade. Regarding estimated test score means for each grade as distinct variables so that an observation consists of a set of estimated means for different grade levels and groups, the observational units in the test score dataset are the districts. The covariate dataset observational units are also the districts.


### Sample sizes

We will count the number of observational units in each dataset. We'll count the number of units in the test dataset and the number of units in the covariate dataset separately, storing the values as `ca_cov_units` and `ca_main_units`, respectively.

In [ ]:
ca_cov_units = ca_cov['sedalea'].nunique()
ca_main_units = ca_main['sedalea'].nunique()

print('units in covariate data: ', ca_cov_units)
print('units in test score data: ', ca_main_units)

### Sampling Design Analysis

We will examine the sampling design characteristics of our datasets by addressing several key aspects of the data collection and scope of inference.

**Population Identification**: The relevant population for the datasets we've imported is California school districts.

**Sample Coverage**: Looking at the proportion of the population captured in our sample (referencing [Fingertip Facts on Education in California](https://www.cde.ca.gov/ds/sd/cb/ceffingertipfacts.asp)), the coverage varies by district type (`High` and `Other`) and by school type (`Intermediate/Middle` or `High`).

**Dataset Classification**: Given that the sampling frame is not clearly identified, we suspect this is administrative data, as the school system is inherently concerned about the success of its students and would likely systematically collect this information.

**Scope of Inference**: In light of these sample characteristics, we can generalize our findings to the California public school system. However, it would be unreasonable to use this data to generalize to the California private school system, as private schools operate under different administrative structures and may not be included in this data collection framework. 



# Data tidying

Since you've already had some guided practice doing this in previous assignments, you'll be left to fill in a little bit more of the details on your own in this assignment. You'll work with the following variables from each dataset:

* **Test score data**
    + District ID
    + District name
    + Grade
    + Test subject
    + Estimated male-female gap
* **Covariate data**
    + District ID
    + Locale
    + Grade
    + Socioeconomic status (all demographic groups)
    + Log median income (all demographic groups)
    + Poverty rate (all demographic groups)
    + Unemployment rate (all demographic groups)
    + SNAP benefit receipt rate (all demographic groups)


### Test score data (Seda Codebook geodist)

| Variable | Description                         |
|----------|-------------------------------------|
| sedalea  | District ID (SEDA LEA ID)                        |
| sedaleaname| District Name                    |
| grade | Tested Grade (g)                    |
| subject | Tested Subject (b)                    |
| cs_mn_mfg     | Geographically Defined District grade-year-subject (gyb) Estimated Male-Female Gap Estimate, Cohort Scale (CS)   |



### Covariate data (Seda Codebook Cov_geodist)

| Variable | Description                         |
|----------|-------------------------------------|
| sedalea  | District ID (SEDA LEA ID) |
| urban    | City/urban locale                   |
| suburb   | Suburban locale                      |
| town     | Town locale                          |
| rural    | Rural locale                          |
| grade | Grade Level                    |
| sesall   | ses composite, eb estimate, all families, time-varying           |
| lninc50all   | log of median income, eb estimate, all families, time-varying       |
| povertyall   | poverty rate, eb estimate, all families, time-varying           |
| unempall   | unemployment rate, eb estimate, all families, time-varying           |
| snapall    | snap receipt rate, eb estimate, all families, time-varying                   |
| perrl    | Percent reduced lunch                |
| perfrl   | Percent free and reduced lunch       |
| perecd   | Percent economically disadvantaged in the grade |

### Variable Selection and Data Exploration

We will identify and extract the variables of interest from our datasets. First, we'll examine the dimensions of our data, then select the relevant columns for our analysis.

After downloading the codebooks from the 'data' directory and reviewing the variable documentation, we'll store the column names of interest in lists named `main_vars` and `cov_vars`.

In [ ]:
print(ca_main.shape)
print(ca_cov.shape)

In [ ]:
# storing variable names of interest
main_vars = ca_main[['sedalea', 'sedaleaname', 'grade', 'subject', 'cs_mn_mfg']].columns  
cov_vars = ca_cov[['sedalea', 'locale', 'grade', 'sesall', 'lninc50all', 'povertyall', 'unempall', 'snapall']].columns

### Data Subsetting

We will now extract only the columns of interest from our datasets using the variable lists we created in the previous step. This will create more focused datasets containing only the variables relevant to our analysis of achievement gaps.

We'll create subsets of both the test score data and covariate data, storing the results as `main_sub` and `cov_sub`, respectively.

In [ ]:
# slice columns to select variables of interest
main_sub = ca_main[main_vars]
cov_sub = ca_cov[cov_vars]

In [ ]:
main_sub.head()

In [ ]:
cov_sub.head()

### Data Merging

We will now combine our test score data with the covariate data to create a comprehensive dataset for analysis. We'll merge the datasets based on both the district ID (`sedalea`) and grade level (`grade`), ensuring that each test score observation is matched with its corresponding district-level covariates.

We'll use a left join to retain all observations from the test score data while adding the relevant covariate information. The resulting merged dataset will be stored as `rawdata`.


In [ ]:
# merge covariates with gap data
rawdata = pd.merge(main_sub, cov_sub, how = 'left', on = ['sedalea','grade'])

# print first four rows
rawdata.head()

### Data Cleaning: Renaming and Reordering Columns

We will now refine our merged dataset by assigning more descriptive column names and organizing them in a logical order for analysis. This will make our data more readable and easier to work with throughout our analysis.

We'll rename the columns to be more intuitive and reorder them to flow from district identifiers to location and socioeconomic variables to our outcome measures (grade, subject, and gender gap).

In [ ]:
# define dictionary mapping for renaming columns
column_name = {"sedalea": "District ID", "sedaleaname": "District", "locale": "Locale", "lninc50all": "log(Median income)",
               "povertyall": "Poverty rate", "unempall": "Unemployment rate", "snapall": "SNAP rate", "sesall": "Socioeconomic index",
               "grade": "Grade", "subject": "Subject", "cs_mn_mfg": "Gender gap"}

# specify order of columns
name_order = ['District ID', 'District', 'Locale', 'log(Median income)','Poverty rate', 
              'Unemployment rate', 'SNAP rate', 'Socioeconomic index', 'Grade', 'Subject', 'Gender gap']

# rename and reorder
rawdata_mod1 = rawdata.rename(columns=column_name)
rawdata_mod1 = rawdata_mod1[name_order]

# print first four rows
rawdata_mod1.head(4)

In [ ]:
rawdata_mod1.shape

### Data Reshaping: Pivoting to Tidy Format

We will now address a data structure issue where our `Gender gap` column contains values for two different variables: math test score gaps and reading test score gaps. To create a tidy dataset where each column represents a single variable, we'll pivot the table to separate these into distinct columns.

We'll use the `.pivot()` method to spread the gender gap values across separate `Math gap` and `Reading gap` columns, with the subject type determining which column receives each value. This transformation will make our data easier to analyze and visualize.

In [ ]:
# pivot to unstack gender gap (fixing tidy issue: multiple variables in one column)
seda_data = rawdata_mod1.pivot(index= rawdata_mod1.columns[:-2], columns = 'Subject',
                  values = 'Gender gap').rename(columns = {'mth' : 'Math gap', 'rla': 'Reading gap'}).reset_index()
seda_data.columns.name = None

# print first four rows
seda_data.head()

### Data Validation: Sanity Check After Tidying

We need to verify that our data reshaping operations did not accidentally lose any observations. This is a crucial quality control step to ensure the integrity of our dataset after the pivoting transformation.

We'll count the number of unique districts in our tidied dataset (`seda_data`) and compare it to the number of unique districts in the original test score data (`ca_main`). These counts should match if our data transformation preserved all observations correctly.



In [ ]:
# number of districts in tidied data compared with raw
data_units = seda_data['District ID'].nunique()
ca_main_units = ca_main['sedalea'].nunique()

if(data_units == ca_main_units):
    print("Tidying was done correctly!")
else:
    print("ERORR: Tidying was not done correctly")

### Missing Data Analysis

We will now examine the extent of missing data in our dataset. Gap estimates were not calculated for certain grades in certain districts due to small sample sizes (insufficient individual test records). Understanding the pattern and extent of missing data is important for interpreting our results and determining appropriate analytical approaches.

We'll investigate missing data at two levels: 
* (i) the proportion of individual observations (rows) that are missing for each gap variable, and 
* (ii) the proportion of districts that have incomplete data for at least one grade level.


In [ ]:
# proportion of missing values
math_missing, reading_missing = seda_data[['Math gap', 'Reading gap']].isna().sum(axis = 0) / seda_data.shape[0]
math_missing = round(math_missing * 100, 2) 
reading_missing = round(reading_missing * 100, 2)


# proportion of districts with missing values
district_missing = seda_data.groupby('District ID')[['Math gap', 'Reading gap']].\
apply(lambda x: x.isnull().any(axis=0)).any(axis=1).mean().item()
district_missing = round(district_missing, 2) * 100
# apply(lambda x: x.isnull().any(axis=0)) : gets all disctrict's grade math gap and reading gap. True means missing.
# .any(axis=1): checks if there is a True in either column of District ID.

print("Percent Math missing:", math_missing, "\nPercent Reading missing:", reading_missing, "\nPercent District Missing", district_missing)

 This analysis reveals that approximately 28% of observations are missing for both math and reading gaps, and about half of all districts have incomplete data for at least one grade level or subject area.


### Investigating Missing Data Patterns in Education Achievement

As part of our investigation into education data quality, we're examining whether certain types of school districts are more likely to have missing information about math and reading achievement gaps. This analysis will help us understand potential data collection disparities and whether certain communities might be systematically underrepresented in education reporting.

We should expect that some locales are more likely to have missing data. It is reasonable to infer that rural areas to be more likely to have missing data, as there is often little communication between rural places and the state government, primarily due to the state government's structure. Most rural areas are "lower income" compared to places such as Los Angeles and San Francisco, so there will be potential bias in that regard.

#### Data Quality Assessment: Missing Achievement Gap Information

We are conducting a **data quality check** on our education dataset to identify missing information about academic achievement gaps across different geographic locations.

#### What We're Analyzing

Our analysis examines:
- **Math gap** and **Reading gap** data - measures of academic achievement disparities
- **Locale categories** - different geographic area types (urban, suburban, rural, etc.)

In [ ]:
seda_data.groupby(['Locale'], observed=False)[['Math gap', 'Reading gap']].apply(lambda x: x.isnull().sum())

Based on the table, our guess was correct. There is a significant amount of missing data from Rural areas. At a glance, they appear to represent half of the missing data for test scores.

Examining the Locales column, we notice a typo for `Sururb, Small.` We will correct this when creating our bar chart to see the disparities in missing test scores by locale.

In [ ]:
# Fix Sururb, Small typo
seda_data['Locale'] = seda_data['Locale'].replace('Sururb, Small', 'Suburb, Small')

In [ ]:
seda_data.head()

In [ ]:
temp_seda = seda_data.copy()
temp_seda['Income bracket'] = pd.cut(np.e**seda_data['log(Median income)'], 8)
result = temp_seda.groupby(['Locale', 'Income bracket'], observed=False)[['Math gap', 'Reading gap']].apply(lambda x: x.isnull().sum())

In [ ]:
temp_seda.groupby(['Income bracket'], observed=False)[['Math gap', 'Reading gap']].apply(lambda x: x.isnull().sum())

In [ ]:
# Get the missing values by locale
missing_by_locale = seda_data.groupby(['Locale'], observed=False)[['Math gap', 'Reading gap']].apply(lambda x: x.isnull().sum())

# Extract the main locale type (before the comma) and reset index
missing_df = missing_by_locale.reset_index()
missing_df['Main Locale'] = missing_df['Locale'].str.split(',').str[0]

# Group by main locale type and sum the missing values
grouped_missing = missing_df.groupby('Main Locale')[['Math gap', 'Reading gap']].sum().reset_index()

# Reshape data to long format for Altair
long_data = pd.melt(grouped_missing, 
                    id_vars=['Main Locale'], 
                    value_vars=['Math gap', 'Reading gap'],
                    var_name='Gap Type', 
                    value_name='Missing Count')

# Alternative: Side-by-side bars
chart = alt.Chart(long_data).mark_bar().encode(
    y=alt.Y('Main Locale:N', title='Locale Type'),
    x=alt.X('Missing Count:Q', title='Number of Missing Values'),
    color=alt.Color('Gap Type:N', legend=alt.Legend(title=None, labelFontSize=12)), # title='Achievement Gap Type'
    yOffset=alt.YOffset('Gap Type:N'),
    tooltip=['Main Locale:N', 'Gap Type:N', 'Missing Count:Q']
).properties(
    width=800,
    height=400,
    title='Missing Values in Achievement Gap Data by Locale Type'
).configure_legend(
    orient='none',
    direction='horizontal',
    legendX = 300,
    legendY = -40,
    padding= 10  # Add padding around legend
)

chart.show()

<!-- END QUESTION -->

# Exploratory graphics

For the purpose of visualizing the relationship between estimated gender gaps and socioeconomic variables, you'll find it more helpful to store a non-tidy version of the data. The cell below rearranges the dataset so that one column contains an estimated gap, one column contains the value of a socioeconomic variable, and the remaining columns record the gap type and variable identity. 

Ensure that your results above match the reference dataset before running this cell.

In [ ]:
# format data for plotting
plot_df = seda_data.melt(
    id_vars = name_order[0:9],
    value_vars = ['Math gap', 'Reading gap'],
    var_name = 'Gap type',
    value_name = 'Gap'
).melt(
    id_vars = ['District ID', 'District', 'Locale', 'Gap type', 'Gap', 'Grade'],
    value_vars = name_order[3:8],
    var_name = 'Socioeconomic variable',
    value_name = 'Measure'
)

# preview
plot_df.head()

## Gender gaps and socioeconomic factors

The cell below generates a panel of scatter plots showing the relationship between the estimated gender gap and socioeconomic factors for all grade levels by test subject. The plot suggests that the reading gap favors girls consistently across the socioeconomic spectrum -- in a typical district, girls seem to outperform boys by 0.25 standard deviations of the national average. By contrast, the math gap appears to depend on socioeconomic factors — boys only seem to outperform girls under *better* socioeconomic conditions.  


In [ ]:
# plot gap against socioeconomic variables by subject for all grades
fig1 = alt.Chart(plot_df).mark_circle(opacity = 0.1).encode(
    y = 'Gap',
    x = alt.X('Measure', scale = alt.Scale(zero = False), title = ''),
    color = 'Gap type'
).properties(
    width = 400,
    height = 200
).facet(
    column = alt.Column('Socioeconomic variable')
).resolve_scale(x = 'independent')

fig1

## Grade-Level Analysis: Examining Relationships Across Educational Stages

We will now investigate whether the relationships between achievement gaps and socioeconomic factors remain consistent across different grade levels. This analysis will help us understand if the patterns we observed in our previous exploration persist throughout students' educational progression or if they vary by academic stage.

We'll develop a comprehensive visualization featuring a 5×5 grid of scatterplots. Each column represents a different socioeconomic variable, while each row depicts a specific grade level. This facetted setup enables us to systematically analyze how the relationship between achievement gaps and socioeconomic factors varies both across different district measures and educational stages.

In [ ]:
base = alt.Chart(plot_df).mark_circle(opacity = 0.1).encode( # https://altair-viz.github.io/user_guide/compound_charts.html
    y = alt.Y('Gap', axis=alt.Axis(orient='left')),
    x = alt.X('Measure', scale = alt.Scale(zero = False), title = ''),
    color = 'Gap type'
)

# trend line  
trend = alt.Chart(plot_df).transform_regression(
    groupby = ['Gap type', 'Grade'],
    on = 'Measure', 
    regression = 'Gap'
).mark_line().encode(
    x = 'Measure',
    y = alt.Y('Gap', axis=alt.Axis(orient='left')),
    color = 'Gap type'
)

# y-axis (just for the right axis)
secondary_axis = alt.Chart(plot_df).mark_point(opacity=0).encode(
    x = alt.X('Measure', scale = alt.Scale(zero = False)),
    y = alt.Y('Gap', axis=alt.Axis(orient='right', title=''))
)

# Combine all layers
fig2 = (base + trend + secondary_axis).properties(
    width = 350,
    height = 150
).facet(
    column = 'Socioeconomic variable',
    row = 'Grade'
).resolve_scale(x = 'independent')

fig2

### Key Observations

* Reading gaps consistently favor girls across all socioeconomic levels and grade levels, with relatively flat gradients showing minimal sensitivity to socioeconomic conditions.

* Math gaps exhibit systematic socioeconomic sensitivity, as socioeconomic status improves (higher income, lower poverty/SNAP rates, higher SES index), math gaps generally shift toward favoring boys or approach parity.

* Patterns are consistently observed across various SES indicators—whether looking at poverty rates, SNAP participation, unemployment, or median income—the directional relationships between socioeconomic status and gender gaps stay stable.

While these various socioeconomic indicators all show similar patterns in gender achievement gaps, median income provides the clearest and simplest way to measure family economic resources. To better understand how socioeconomic status affects gender gaps in academic achievement, we will focus our detailed analysis on income differences across grade levels.


In [ ]:
base = alt.Chart(plot_df).mark_circle(opacity = 0.1).transform_filter(
    alt.datum['Socioeconomic variable'] == 'log(Median income)'
).encode(
    y = alt.Y('Gap', axis=alt.Axis(orient='left')),
    x = alt.X('Measure', scale = alt.Scale(zero = False), title = ''),
    color = 'Gap type'
)

# trend line  
trend = alt.Chart(plot_df).transform_filter(
    alt.datum['Socioeconomic variable'] == 'log(Median income)'
).transform_regression(
    groupby = ['Gap type', 'Grade'],
    on = 'Measure', 
    regression = 'Gap'
).mark_line().encode(
    x = 'Measure',
    y = alt.Y('Gap', axis=alt.Axis(orient='left')),
    color = 'Gap type'
)

# y-axis (just for the right axis)
secondary_axis = alt.Chart(plot_df).mark_point(opacity=0).transform_filter(
    alt.datum['Socioeconomic variable'] == 'log(Median income)'
).encode(
    x = alt.X('Measure', scale = alt.Scale(zero = False)),
    y = alt.Y('Gap', axis=alt.Axis(orient='right', title=''))
)

# Combine all layers
fig2 = (base + trend + secondary_axis).properties(
    width = 350,
    height = 250
).facet(
    column = 'Grade'  # Removed column faceting since we're only showing one socioeconomic variable
).resolve_scale(x = 'independent')

fig2


### Math Gap Pattern

* Variable direction: The math gap fluctuates, sometimes favoring girls, sometimes boys, or showing parity, depending on grade level and income level.

* Consistent income gradient: The slope stays the same across all levels — as income rises, the gap usually shifts in favor of boys (or toward equality).

* Variable baseline gap: The starting point at low-income levels varies widely by grade, ranging from parity (grade 4) to significant gaps favoring girls (grades 6 and 8).

Grade-specific pattern:

* Grade 4: Gender parity at low-income levels, but a gap emerges favoring boys as income increases.

* Grade 5: Initial gap slightly favors girls, though not significantly, converges to parity around middle-income levels, then shifts to favor boys

* Grade 6: Large initial gap favoring girls, equalizes by middle-income, slight boy advantage at high-income.

* Grade 7: Moderate initial gap favoring girls, then converges to parity by middle-income levels, with minimal differences at high income.

* Grade 8: Substantial initial gap favoring girls, maintaining a slight girl advantage through middle income levels, then converging to parity at high income.
  
While the income gradient stays consistent across grades 4-8, the baseline gender gap at low-income levels varies significantly by grade.

#### Takeaway

1. **Highly sensitive to socioeconomic status** (significant income gradient effect)
2. **Gender advantage varies** across different grade and income levels.
3. **Baseline gap varies** (different baseline gaps at each grade level)

### Reading Gap Pattern

* Consistent trend: Girls outperform boys in reading at all grade and income levels.

* Flat income gradient: The reading gap stays fairly consistent across different income levels in each grade.

* Widening baseline gap: The size of the gap favoring girls grows steadily from grade 4 to grade 8.


Grade-speific pattern:

* Grade 4: Modest reading gap favoring girls.

* Grade 5: Gap widens slightly compared to grade 4.

* Grades 6-8: Gap continues to widen, with grade 8 showing the largest reading advantage for girls.

The reading gap favors girls throughout all grade levels. The income gradient stays consistent across all grade levels; however the basleine gender gap at all income levels widens after each grade level, favoring girls.

#### Takeaway

While the gender gap in math is highly sensitive to socioeconomic status and can change in direction, the gender gap in reading is: 

1. **Stable across income levels** (no income gradient) 
2. **Consistently favors girls** at all income levels 
3. **Grows larger with age/grade level** (widening baseline gap)


## Creating District-Level Achievement Data

To analyze patterns across school districts rather than individual grades, we need to consolidate our grade-level data into district-level summaries. This aggregation will allow us to examine how achievement gaps vary between districts while incorporating important socioeconomic characteristics.

### Data Transformation Process

Our original dataset contains multiple records per district (one for each grade level tested). To create a comprehensive district-level analysis, we're computing average achievement gaps across all grades within each district while preserving the socioeconomic variables that characterize each district.

### Methodology

Since socioeconomic variables like median income and locale type are consistent across all grades within a district, we can safely aggregate the achievement data while maintaining these important contextual variables.


In [ ]:
seda_data_agg = seda_data.drop_duplicates(subset=['District ID']).reset_index().drop(columns='index')
seda_data_agg['Math gap'] = seda_data.groupby(['District ID'])['Math gap'].mean().values
seda_data_agg['Reading gap'] = seda_data.groupby(['District ID'])['Reading gap'].mean().values
seda_data_agg.head()

### Reshaping Data for Visualization

To create effective visualizations comparing math and reading achievement gaps, we need to restructure our district-level data from a "wide" format (separate columns for Math gap and Reading gap) to a "long" format where both gap types are stacked in a single column.

### Data Transformation

This reshaping process, known as "melting," will allow us to easily create comparative plots and analyze both subjects together using visualization libraries.


In [ ]:
# format data for plotting
agg_plot_df = seda_data_agg.melt(
    id_vars = ['District ID','log(Median income)',
       'Poverty rate', 'Unemployment rate', 'SNAP rate', 'Socioeconomic index'],
    value_vars = ['Math gap', 'Reading gap'],
    var_name = 'Subject',
    value_name = 'Gap'
).rename(columns={'Gap':'Average estimated gap'})

agg_plot_df.head()

### District average gaps

We will construct a scatterplot of the average estimated gap against log(Median income) by subject for each district and add trend lines.


There is a gap but only in math. As median increases, the math gap increases as well. The reading gap stays consistent throughout median income.

In [ ]:
# scatterplot
scatter = alt.Chart(agg_plot_df).transform_filter(
    alt.FieldOneOfPredicate(field = 'Subject',
                            oneOf = ['Math gap', 'Reading gap'])
).mark_circle(opacity = 0.5).encode(
    y = alt.Y('Average estimated gap', scale = alt.Scale(zero = False)),
    x = alt.X('log(Median income)', scale = alt.Scale(zero = False)),
    color = alt.Color('Subject', legend=alt.Legend(title=None, labelFontSize=14))
)

# trend line
trend = scatter.transform_regression(
    groupby = ['Subject'],
    on = 'log(Median income)', 
    regression = 'Average estimated gap'
).mark_line(color = 'black')

# combine layers
fig4 = (scatter + trend).properties(
    width = 800,
    height = 400,
    title="Math and Reading Gap by Income"
).configure_legend(
    orient='none',
    direction='horizontal',
    legendX = 300,
    legendY = -40,
    padding= 10  # Add padding around legend
)

# display
fig4

## Introducing Income Bracket

Now let's try to capture this pattern in *tabular* form. The cell below adds an `Income bracket` variable by cutting the median income into 8 contiguous intervals using `pd.cut()`, and tabulates the average socioeconomic measures and estimated gaps across districts by income bracket. Notice that with respect to the gaps, this displays the pattern that is shown visually in the figures above. 

In [ ]:
seda_data_agg = seda_data.drop_duplicates(subset=['District ID']).reset_index().drop(columns='index')
seda_data_agg['Math gap'] = seda_data.groupby(['District ID'])['Math gap'].mean().values
seda_data_agg['Reading gap'] = seda_data.groupby(['District ID'])['Reading gap'].mean().values

In [ ]:
seda_data_agg['Income bracket'] = pd.cut(np.e**seda_data_agg['log(Median income)'], 8)
seda_data_agg.groupby('Income bracket', observed=False).mean(numeric_only=True).drop(columns = ['District ID', 'log(Median income)'])

### Analyzing Gender Achievement Gaps in Mathematics Across Income Levels

As part of our investigation into education equity, we're examining whether math achievement gaps between boys and girls vary systematically across different income brackets. This analysis will help us understand if socioeconomic factors influence gender-based academic performance differences.

This analysis helps us determine whether socioeconomic factors affect gender-based differences in academic performance. This approach differs significantly from our previous grade-by-grade analysis in several key ways: while our earlier study used continuous income measures to track how gaps changed along income levels within each specific grade (4-8), this analysis combines math performance across all grades to produce an overall district-level gap score. It then groups districts into distinct income brackets and classifies gap outcomes as favoring boys, girls, or neither. Instead of focusing on slopes and baseline differences across developmental stages, we are now exploring whether entire school districts in different economic contexts exhibit systematically different patterns in their overall gender achievement gaps.

**Research focus:** What proportion of school districts in each income bracket show math achievement gaps that favor boys? Do wealthier or poorer districts show different patterns in gender-based math performance gaps?

### Data Preparation and Analysis

We'll create categories to identify districts where boys significantly outperform girls in math (more than 0.1 standard deviations above the national average), and then examine how these patterns distribute across income levels.


In [ ]:
# Drop rows with missing Math gap values before creating categories
seda_data_agg = seda_data_agg.dropna(subset=['Math gap'])

# define indicator
seda_data_agg = seda_data_agg.assign(
    **{
        'Math gap favoring boys': seda_data_agg['Math gap'] > 0.1,
        'Math gap favoring girls': seda_data_agg['Math gap'] < -0.1,
        'Math gap favoring neither': seda_data_agg['Math gap'].between(-0.1, 0.1, inclusive='both')
    }
)

# proportion of districts with gap favoring boys, by income bracket
income_bracket_boys_favored = seda_data_agg.groupby('Income bracket', observed=True).agg({
    'Math gap favoring boys' : ['count', 'sum', 'mean'], # Get total districts per income bracket, count how many districts favor boys per income bracket, percent that favor boys
    'Math gap favoring girls': ['sum', 'mean'], # Districts favoring girls, percent favoring girls
    'Math gap favoring neither': ['sum', 'mean'] # Districts favoring neither, percent favoring neither
}) #.round(2)

income_bracket_boys_favored.columns = ['Total Districts', 'Districts Favoring Boys', 'Percent Favoring Boys', 
'Districts Favoring Girls', 'Percent Favoring Girls', 
'Districts Favoring Neither', 'Percent Favoring Neither']

# Round only the percentage columns at the end
percentage_columns = ['Percent Favoring Boys', 'Percent Favoring Girls', 'Percent Favoring Neither']
income_bracket_boys_favored[percentage_columns] = income_bracket_boys_favored[percentage_columns].round(3)

income_bracket_boys_favored.reset_index(inplace=True)

# print result
income_bracket_boys_favored

In [ ]:
chart1_data = income_bracket_boys_favored.melt(
    id_vars=['Income bracket'],
    value_vars= income_bracket_boys_favored.columns[1:], # ['Total Districts', 'Districts Favoring Boys', 'Percent Favoring Boys'], 
    var_name='Category', # 'Category'
    value_name='Metric'
)

# print(chart1_data.head())

In [ ]:
# Filter to only the percentage data
# Filter to only the percentage data (excludes raw counts)
percent_data = chart1_data[chart1_data['Category'].str.contains('Percent')].copy()
percent_data['Income bracket'] = percent_data['Income bracket'].astype(str)

# Define income bracket order from lowest to highest income
income_order = [
    '(21980.176, 46455.372]',
    '(46455.372, 70736.321]', 
    '(70736.321, 95017.269]',
    '(95017.269, 119298.218]',
    '(119298.218, 143579.167]',
    '(143579.167, 167860.115]',
    '(167860.115, 192141.064]',
    '(192141.064, 216422.013]'
]

#### Visualization: Gender Achievement Patterns by Income Level

We'll create a horizontal stacked bar chart to visualize how gender-based math achievement gaps distribute across different income brackets.

In [ ]:
# Math Gap by Income Bracket - Horizontal Stacked Bar Chart

# Create base chart object
base = alt.Chart(percent_data)

# Create horizontal stacked bar chart
bars = base.mark_bar().encode(
    y=alt.Y('Income bracket:O', sort=income_order, title=None, 
    axis=alt.Axis(
            ticks=False,     # Hide y-axis ticks
            grid=False,      # Hide grid lines
            title=None       # Hide y-axis title
        )),
    x=alt.X('Metric:Q', 
        scale=alt.Scale(domain=[0, 1]),
        axis=None
    ),
    # Color scheme
    color=alt.Color('Category:N', 
                   scale=alt.Scale(scheme='set2'), # https://vega.github.io/vega/docs/schemes/
                   #scale=alt.Scale(range=['#4A90E2', '#F5A623', '#7B68EE']),
                   legend=alt.Legend(title=None, labelFontSize=12)),
    order=alt.Order('Category:N', sort='ascending')
)

# Add percentage labels positioned at center of each bar segment
text = base.mark_text(
    align='center',
    baseline='middle', 
    fontSize=11,
    color='white',
    fontWeight='bold'
).encode(
    y=alt.Y('Income bracket:O', sort=income_order),
    x=alt.X('x_position:Q', scale=alt.Scale(domain=[0, 1])),
    text=alt.Text('Metric:Q', format='.0%'),
    order=alt.Order('Category:N', sort='ascending')
).transform_window(
    # Calculate cumulative sum for proper stacking
    cumulative_sum='sum(Metric)',
    groupby=['Income bracket'],
    sort=[{'field': 'Category', 'order': 'ascending'}]
).transform_calculate(
    # Position text at center of each segment
    x_position='datum.cumulative_sum - datum.Metric/2'  
).transform_filter(
    # Only show labels for segments > 3% to avoid clutter
    alt.datum.Metric > 0.03  
)

# Combine bars and text labels
chart = (bars + text).resolve_scale(
    color='independent'
).properties(
    width=750,
    height=400,
    title="Math Gap by Income Bracket"
).configure_legend(
    orient='none',
    direction='horizontal',
    legendX = 120,
    legendY = -40,
    padding= 10  # Add padding around legend
)
# Make Ineractive search 
chart

### Statewide averages



In [ ]:
# statewide average
avg = np.array([seda_data_agg['Math gap'].mean(), seda_data_agg['Reading gap'].mean()])
d = {'Math gap': [avg[0]], 'Reading gap': [avg[1]]}
df = pd.DataFrame(data=d)
state_avg = df

# statewide average
state_avg = seda_data_agg.loc[:, ['Reading gap', 'Math gap']].mean() 


# proportion of districts in the state with a math gap favoring boys
math_boys_proportion = seda_data_agg['Math gap favoring boys'].mean()

# proportion of districts in the state with a math gap favoring girls
seda_data_agg['Math gap favoring girls'] = np.where(seda_data_agg['Math gap'] < -0.1, True, False)
math_girls_proportion = seda_data_agg['Math gap favoring girls'].mean()

In [ ]:
state_avg

In [ ]:
math_boys_proportion

In [ ]:
math_girls_proportion

# Conclusion and Limitations

Take a moment to review and reflect on your findings and consider what you have learned from the analysis.